# 🚗 Vehicle Fleet Performance Analysis
This notebook demonstrates a defensively-written, reproducible workflow for loading, cleaning, exploring, and exporting fleet telemetry data.

Notes: keep your raw dataset in `data/` (e.g. `data/vehicle_fleet_data.xlsx`). The notebook includes safe checks so cells can be re-run in any order.

## 1. Setup & Environment
Make sure you have the pinned dependencies installed (see repository `requirements.txt`). This notebook assumes a Python virtual environment is active.

If you are missing packages, run: `python -m pip install -r requirements.txt`

In [ ]:
# Imports and basic utilities
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

In [ ]:
# Safe data loading with fallbacks
data_path = Path('..') / 'data' / 'vehicle_fleet_data.xlsx'
sample_path = Path('..') / 'data' / 'sample_vehicle_data.csv'

def load_data():
    if data_path.exists():
        return pd.read_excel(data_path)
    if sample_path.exists():
        return pd.read_csv(sample_path)
    # minimal synthetic fallback so notebook runs for demos
    return pd.DataFrame({
        'Vehicle ID': ['V1','V2','V3'],
        'Model': ['M1','M2','M1'],
        'Brand': ['A','B','A'],
        'Month': ['2021-01','2021-02','2021-03'],
        'Mileage (km)': [100, 200, 150],
        'Fuel Used (L)': [5, 10, 7],
        'Maintenance Cost (€)': [50, 80, 60],
        'Total Trips': [2,4,3],
        'Start_Station': ['S1','S2','S1'],
        'End_Station': ['E1','E2','E1']
    })

df = load_data()
print('Data loaded — rows:', len(df))
display(df.head())

In [ ]:
# Basic validation and safe feature engineering
required = ['Mileage (km)', 'Fuel Used (L)']
missing = [c for c in required if c not in df.columns]
if missing:
    print('Warning — missing expected columns:', missing)

df = df.copy()
# Defensive arithmetic
df['Fuel Used (L)'] = pd.to_numeric(df.get('Fuel Used (L)', 0), errors='coerce').fillna(0)
df['Mileage (km)'] = pd.to_numeric(df.get('Mileage (km)', 0), errors='coerce').fillna(0)
df['Total Trips'] = pd.to_numeric(df.get('Total Trips', np.nan), errors='coerce')
df['Fuel Efficiency (km/L)'] = df.apply(lambda r: r['Mileage (km)'] / r['Fuel Used (L)'] if r['Fuel Used (L)']>0 else np.nan, axis=1)
df['Cost per km (€)'] = df.apply(lambda r: (r.get('Maintenance Cost (€)',0) / r['Mileage (km)']) if r['Mileage (km)']>0 else np.nan, axis=1)

display(df.head())

In [ ]:
# Summary by Model with graceful handling if column absent
group_col = 'Model' if 'Model' in df.columns else df.columns[0]  # fallback
summary = df.groupby(group_col).agg({
    'Mileage (km)': 'sum' if 'Mileage (km)' in df.columns else 'count',
    'Fuel Used (L)': 'sum' if 'Fuel Used (L)' in df.columns else 'sum',
    'Maintenance Cost (€)': 'sum' if 'Maintenance Cost (€)' in df.columns else 'sum',
}).reset_index()
display(summary.head())

In [ ]:
# Plots (wrapped in try/except so notebook is robust)
try:
    if 'Fuel Efficiency (km/L)' in df.columns:
        plt.figure(figsize=(8,4))
        sns.barplot(x='Model', y='Fuel Efficiency (km/L)', data=df.groupby('Model')['Fuel Efficiency (km/L)'].mean().reset_index())
        plt.title('Average Fuel Efficiency by Model')
        plt.xticks(rotation=45)
        plt.show()
except Exception as e:
    print('Plot skipped (error):', e)

In [ ]:
# Export summary (create folder if needed)
out_dir = Path('..') / 'reports'
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'monthly_summary.xlsx'
try:
    summary.to_excel(out_path, index=False)
    print('Exported summary to', out_path)
except Exception as e:
    print('Export failed:', e)